In [ ]:
import showdown
import agent
import asyncio
import torch
import json
from IPython.display import clear_output

In [ ]:
async def training_loop(agent, showdown, numBattles=5000):
    agentWins = 0
    agentLosses = 0
    cumulativeRewards = 0
    for battle in range(numBattles):
        battleReward = -5
        battleProgress = []
        
        
        await showdown.restart()
        battleDone = False
        totalReward = 0
        
        while not battleDone:
            state = showdown.getState()
            
            validActions = showdown.getValidActions()
            action = agent.act(state, validActions)
            nextState, reward, battleDone, winner = await showdown.executeAction(action)
            print(f"Action: {action}, Reward: {reward}")
            
            battleProgress.append(state, action, reward, nextState, battleDone)
            agent.replay()
            totalReward += reward
            cumulativeRewards += reward
            
            if battleDone:
                print(f"Battle {battle} done. Total reward: {totalReward}")
                if winner == 1:
                    agentWins += 1 
                    battleReward = 10
                else:
                    agentLosses += 1
        
        # Once the battle's done...
        for i, (state, action, reward, nextState, done) in enumerate(battleProgress):
            # Adjust the reward based on how the battle went.
            adjustedReward = reward + (battleReward / (len(battleProgress)))
            agent.remember(state, action, adjustedReward, nextState, done)
        
        agent.replay()        
        
        if battle % 10 == 0 and battle > 0:
            clear_output(wait=True)
            print(f"Cleared Output! Current Stats: \n Wins: {agentWins} \n Losses: {agentLosses} \n Cumulative Rewards: {cumulativeRewards} \n Battles: {battle} \n Epsilon: {agent.epsilon}")
        if battle % 50 == 0 and battle > 0:
            agent.saveModel(f"data/models/model_{battle}.pt")
            agent.saveMemory(f"data/memory/memory_{battle}.json")
            f = open(f"data/stats/{battle}.json", "w")
            f.write(json.dumps({"wins": agentWins, "losses": agentLosses, "rewards": cumulativeRewards}))
            f.close()
            
            # Reset Epsilon
            agent.epsilon = max(agent.epsilon, 0.3)
            
        
        

IndentationError: expected an indented block after 'for' statement on line 36 (2546821136.py, line 39)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
stateSize = 671

possibleActions = json.load(open("data/possible_actions.json", "r"))
actionSize = len(possibleActions)

newAgent = agent.Agent(stateSize, actionSize, device, possibleActions)
newAgent.loadModel("data/models/model_450.pt")
newAgent.loadMemory("data/memory/memory_450.json")

showdown = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-0", "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")

await showdown.connectNoSecurity()
await training_loop(newAgent, showdown, 10000)

In [ ]:
# sd = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "thisisatest12345", "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")
# tasks = set()
# async with asyncio.TaskGroup() as tg:
#     for i in range(1):
#         sd = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-"+str(i), "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle")
#         task = tg.create_task(sd.run())
#         tasks.add(task)

# await asyncio.gather(*tg.tasks)

# await sd.run()
